In [1]:
# import particles 
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Data Simulation

In [2]:
def next_angle(q, u):
    # q_init: array (2,) containing the inital angles
    # u:      array (2,) containing the change rate
    return q + u

def current_position(q, l):
    # q: array (2,) containing the two angles
    # l: array (2,) containing the limb lengths 
    x = l[0] * np.cos(q[0]) + l[1] * np.cos(q[0] + q[1])
    y = l[0] * np.sin(q[0]) + l[1] * np.sin(q[0] + q[1])
    return np.array([x, y])


In [ ]:
n_tasks = 1000
n_episode = 150
n_timesteps = 10

inputs_ = list()
targets = list()
# generate limb length for each task
lens = np.random.normal(loc=1, scale=np.sqrt(0.3), size=(n_tasks, 2))
# for each task, generate {n_episode} episodes, each contains {n_timesteps} timesteps
for i in range(n_tasks):
    limb_length = lens[i]
    q = (np.random.rand(2) * 2 - 1) * np.pi # observation, the angle, uniform [-pi, pi]
    inputs_.append(q)
    for _ in range(n_episode):
        u = np.random.randn(2) # action, the change rate of angles
        for _ in range(n_timesteps):
            q = next_angle(q, u)
    position = current_position(q, limb_length)
    targets.append(position)

inputs_ = np.stack(inputs_, axis=0)
targets = np.stack(targets, axis=0)

assert inputs_.shape == targets.shape # their shape should be both (n_tasks*n_episode*n_timesteps, 2)

# Data simulation 2 (another comprehension)

In [ ]:
n_tasks = 1
n_episode = 150
n_timesteps = 10

inputs_ = list()
targets = list()
# generate limb length for each task
lens = np.random.normal(loc=1, scale=np.sqrt(0.3), size=(n_tasks, 2))
# for each task, generate {n_episode} episodes, each contains {n_timesteps} timesteps
for i in range(n_tasks):
    limb_length = lens[i]
    for _ in range(n_episode):
        q = (np.random.rand(2) * 2 - 1) * np.pi # observation, the angle, uniform [-pi, pi]
        for _ in range(n_timesteps):
            inputs_.append(q)
            u = np.random.randn(2) # action, the change rate of angles
            q = next_angle(q, u)
            position = current_position(q, limb_length)
            targets.append(position)

inputs_ = np.stack(inputs_, axis=0)
targets = np.stack(targets, axis=0)

assert inputs_.shape == targets.shape # their shape should be both (n_tasks*n_episode*n_timesteps, 2)

# Split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs_, targets, test_size=0.1, random_state=666)

# Build model

In [ ]:
input_length = 2
latent_dim = 512
output_length = 2

inputs = Input(shape=(input_length,))
x = Dense(units=latent_dim, activation='relu')(inputs)
# x = Dropout(rate=0.5)(x)
x = Dense(units=latent_dim, activation='relu')(x)
outputs = Dense(units=output_length, activation='relu')(x)
model = Model(inputs=inputs, outputs=outputs, name="Two-Link-Arm") # no space in the name!

In [ ]:
model.summary()

Model: "Two-Link-Arm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               384       
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 258       
Total params: 17,154
Trainable params: 17,154
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.losses.MeanAbsoluteError()])
            
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/100
10547/10547 [==============================] - 37s 4ms/step - loss: 1.2069 - mean_absolute_error: 0.8734 - val_loss: 1.2007 - val_mean_absolute_error: 0.8714
Epoch 2/100
10547/10547 [==============================] - 37s 3ms/step - loss: 1.1953 - mean_absolute_error: 0.8681 - val_loss: 1.1986 - val_mean_absolute_error: 0.8702
Epoch 3/100
10547/10547 [==============================] - 38s 4ms/step - loss: 1.1940 - mean_absolute_error: 0.8675 - val_loss: 1.1973 - val_mean_absolute_error: 0.8694
Epoch 4/100
10547/10547 [==============================] - 38s 4ms/step - loss: 1.1927 - mean_absolute_error: 0.8668 - val_loss: 1.1966 - val_mean_absolute_error: 0.8690
Epoch 5/100
10547/10547 [==============================] - 38s 4ms/step - loss: 1.1920 - mean_absolute_error: 0.8664 - val_loss: 1.1962 - val_mean_absolute_error: 0.8685
Epoch 6/100
10547/10547 [==============================] - 38s 4ms/step - loss: 1.1916 - mean_absolute_error: 0.8662 - val_loss: 1.1952 - val_mean_abs

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.losses.MeanAbsoluteError()])
            
model.fit(x_train, y_train, epochs=500, batch_size=16, validation_data=(x_test, y_test))

Epoch 1/500
57/57 [==============================] - 0s 4ms/step - loss: 1.3826 - mean_absolute_error: 0.9484 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 2/500
57/57 [==============================] - 0s 3ms/step - loss: 1.3820 - mean_absolute_error: 0.9432 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 3/500
57/57 [==============================] - 0s 3ms/step - loss: 1.3820 - mean_absolute_error: 0.9480 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 4/500
57/57 [==============================] - 0s 3ms/step - loss: 1.3820 - mean_absolute_error: 0.9474 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 5/500
57/57 [==============================] - 0s 3ms/step - loss: 1.3820 - mean_absolute_error: 0.9554 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 6/500
57/57 [==============================] - 0s 2ms/step - loss: 1.3820 - mean_absolute_error: 0.9482 - val_loss: 1.3978 - val_mean_absolute_error: 0.9682
Epoch 7/500
57/57 [===

KeyboardInterrupt: 